# 4. LangChain 기초


In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 4.1. LangChain 개요


### LangChain 설치


In [4]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0

### LangSmith 설정


In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 4.2. LLM / Chat model


### LLM


In [8]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("안녕하세요")
print(ai_message)

. 저는 김민지입니다. 만나서 반가워요. 저는 대학교에서 경영학을 전공하고 있어요. 취미는 음악 감상이고, 특히 팝 음악을 좋아해요. 또한 여행을 좋아해서 여러 나라를 다녀보는 것이 꿈이에요. 앞으로 잘 부탁드립니다!


### Chat model


In [9]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요! 저는 존이라고 합니다!"),
    AIMessage(content="안녕하세요, 존님! 어떤 도움이 필요하신가요?"),
    HumanMessage(content="제 이름을 아시나요?"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

네, 존님이라고 말씀하셨습니다! 어떻게 도와드릴까요?


### 스트리밍


In [10]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요!"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

## 4.3. Prompt template


### PromptTemplate


In [11]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리의 레시피를 생각해 주세요.

요리명: {dish}""")

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 카레


#### ＜보충: 프롬프트 변수가 1개인 경우＞


In [12]:
prompt_value = prompt.invoke("카레")
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 카레


### ChatPromptTemplate


In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder


In [14]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요! 저는 존이라고 합니다!"),
            AIMessage("안녕하세요, 존님! 어떻게 도와드릴까요?"),
        ],
        "input": "제 이름을 아시나요?",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 저는 존이라고 합니다!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 존님! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='제 이름을 아시나요?', additional_kwargs={}, response_metadata={})]


### LangSmith의 Prompts


In [16]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("ychoi/recipe")

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


### (칼럼) 멀티모달 모델의 입력 처리


In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "이미지를 설명해 주세요."},
                {"type": "image_url", "image_url": {"url": "{image_url}"}},
            ],
        ),
    ]
)
image_url = "https://raw.githubusercontent.com/ychoi-kr/langchain-book/main/cover.jpg"

prompt_value = prompt.invoke({"image_url": image_url})

In [20]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

이 이미지는 책 표지입니다. 제목은 "챗GPT와 랭체인을 활용한 LLM 기반 AI 앱 개발"입니다. 표지에는 벌 모양의 종이접기 도안이 그려져 있으며, 노란색과 파란색 줄무늬가 특징입니다. 책은 랭체인과 LLM 애플리케이션 구축에 관한 내용을 다루고 있는 것으로 보입니다.


## 4.4. Output parser


### PydanticOutputParser를 사용한 Python 객체 변환


In [21]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [22]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [23]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [24]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [25]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [26]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "감자 2개",
    "당근 1개",
    "카레 가루 3큰술",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 4컵"
  ],
  "steps": [
    "양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.",
    "냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.",
    "닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.",
    "감자와 당근을 추가하고 함께 볶습니다.",
    "카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.",
    "끓기 시작하면 중약불로 줄이고 20분간 끓입니다.",
    "소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.",
    "밥과 함께 서빙합니다."
  ]
}


In [27]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.', '냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.', '감자와 당근을 추가하고 함께 볶습니다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.', '끓기 시작하면 중약불로 줄이고 20분간 끓입니다.', '소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.', '밥과 함께 서빙합니다.']


### StrOutputParser


In [28]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요. 저는 AI 어시스턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요. 저는 AI 어시스턴트입니다.


## 4.5. Chain—LangChain Expression Language(LCEL) 개요


### prompt와 model 연결


In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [30]:
chain = prompt | model

In [31]:
ai_message = chain.invoke({"dish": "카레"})
print(ai_message.content)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 돼지고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 코코넛 밀크 또는 물 2컵
- 식용유 2 큰술
- 소금, 후추 약간
- 마늘 2쪽 (다진 것)
- 생강 1 작은 조각 (다진 것)
- 선택 재료: 피망, 버섯, 완두콩 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.

3. **마늘과 생강 추가**: 다진 마늘과 생강을 넣고 향이 날 때까지 볶습니다.

4. **고기 추가**: 손질한 고기를 넣고 겉면이 익을 때까지 볶습니다.

5. **채소 추가**: 감자와 당근을 넣고 함께 볶습니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 이때 향신료의 향이 나기 시작합니다.

7. **액체 추가**: 코코넛 밀크 또는 물을 부어주고, 소금과 후추로 간을 맞춥니다.

8. **끓이기**: 끓어오르면 불을 줄이고 뚜껑을 덮고 20-30분 정도 끓입니다. 중간에 저어주면서 바닥이 타지 않도록 합니다.

9. **마무리**: 채소가 부드러워지면 불을 끄고, 필요에 따라 추가 간을 맞춥니다.

10. **서빙**: 밥이나 난과 함께 따뜻하게 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 채소나 해산물을 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### StrOutputParser를 연결에 추가


In [32]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "카레"})
print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료가 있다면 함께 손질합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣어 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 선택 재료가 있다면 이때 추가합니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### PydanticOutputParser를 사용한 연결


In [33]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [35]:
chain = prompt_with_format_instructions | model | output_parser

In [36]:
recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.', '냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.', '감자와 당근을 추가하고 함께 볶습니다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.', '끓기 시작하면 중약불로 줄이고 20분간 끓입니다.', '소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.', '밥과 함께 따뜻하게 서빙합니다.']


### （칼럼）with_structured_output


In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['카레 가루', '닭고기 또는 소고기', '양파', '당근', '감자', '마늘', '생강', '식용유', '소금', '후추', '물', '코코넛 밀크 (선택사항)', '채소 (선택사항)'] steps=['양파, 마늘, 생강을 다져서 준비한다.', '식용유를 팬에 두르고 다진 양파를 넣어 볶는다.', '양파가 투명해지면 다진 마늘과 생강을 넣고 볶는다.', '고기를 넣고 겉면이 익을 때까지 볶는다.', '당근과 감자를 깍둑썰기하여 넣는다.', '카레 가루를 넣고 잘 섞는다.', '물과 소금을 넣고 끓인다.', '중불에서 약 20분 동안 끓인다. (재료가 부드러워질 때까지)', '원하는 경우 코코넛 밀크를 추가하여 크리미하게 만든다.', '후추로 간을 맞추고, 필요에 따라 추가 채소를 넣는다.', '완성된 카레를 밥과 함께 서빙한다.']


## 4.6.LangChain의 RAG 관련 컴포넌트


### Document loader


In [38]:
!pip install langchain-community==0.3.0 GitPython==3.1.43

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.44
    Uninstalling GitPython-3.1.44:
      Successfully uninstalled GitPython-3.1.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0


In [39]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

402


### Document transformer


In [40]:
!pip install langchain-text-splitters==0.3.0

  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 0.3.0 which is incompatible.


In [41]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

1409


### Embedding model


In [42]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [43]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.006380176637321711, 0.008416633121669292, 0.023549241945147514, -0.027167202904820442, 0.03552967682480812, 0.006282686721533537, -0.0015246346592903137, 0.01739654317498207, 0.011503815650939941, -0.02567235752940178, -0.0010080733336508274, -0.0023573613725602627, -0.009467358700931072, -0.008736183866858482, -0.015175937674939632, 0.04878830909729004, 0.007517559453845024, -0.015067615546286106, -0.02918199636042118, 0.0004884653608314693, 0.02539071999490261, 0.03301659971475601, -0.026517270132899284, 0.05056479573249817, 0.025889001786708832, -0.0474017858505249, -0.01745070330798626, 0.03472809121012688, -0.013106984086334705, -0.10693565756082535, -0.013009494170546532, -0.054507721215486526, -0.02539071999490261, 0.0327782928943634, -0.015305924229323864, 0.05255792289972305, 0.04566863179206848, 0.009694835171103477, -0.011471319012343884, -0.019942114129662514, 0.012294567190110683, -0.004083746112883091, 0.015305924229323864, 0.004265185911208391, 0.0301352310925722

### Vector store


In [44]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.6 MB/s eta 0:00

In [45]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

In [46]:
retriever = db.as_retriever()

In [47]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) service that automaticall

### LCEL을 사용한 RAG Chain 구현


In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 바탕으로 질문에 답변해 주세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [49]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

네, AWS S3에서 데이터를 읽어 들이기 위한 Document loader가 있습니다. `S3DirectoryLoader`와 `S3FileLoader`를 사용할 수 있습니다.
